In [1]:
import pandas as pd

from xgboost import XGBRegressor

Загружаем данные

In [2]:
train_pool = pd.read_csv('train.csv', encoding='utf-8')
test_pool = pd.read_csv('test.csv', encoding='utf-8')

Напишем функции, генерирующие простые признаки основанные на пересечении триграмм между запросом и названием организации

In [3]:
def get_trigrams(string):
    string = '^^' + string + '$$'
    trigrams = set()
    trigrams_count = 0
    
    for i in range(len(string) - 2):
        trigrams.add(string[i:i+3])
        trigrams_count += 1
        
    return trigrams, trigrams_count

def common_trigrams_factors(query, org_name):
    query_trigrams, query_trigrams_count = get_trigrams(query)
    org_name_trigrams, org_name_trigrams_count = get_trigrams(org_name)

    factors = [float(len(query_trigrams.intersection(org_name_trigrams)))]

    factors.append(0. if query_trigrams_count == 0. else 0.1 + factors[0] / query_trigrams_count)
    factors.append(0. if org_name_trigrams_count == 0. else 0.1 + factors[0] / org_name_trigrams_count)    
    
    return factors

Посчитаем данные факторы для каждого файла

In [4]:
def calc_trigram_factors(row):
    return pd.Series(common_trigrams_factors(row.query, row.org_name))
    
train_factors = train_pool.apply(calc_trigram_factors, axis=1)
test_factors = test_pool.apply(calc_trigram_factors, axis=1)

Обучаем модель

In [5]:
clf = XGBRegressor(n_estimators=100)
clf.fit(train_factors.values, train_pool[['relevance']].values)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

Добавляем колонку с предсказанной релевантностью

In [6]:
test_pool['relevance'] = clf.predict(test_factors.values)

Сортируем организации по предсказанной релевантности для каждой организации и записываем их в файл

In [7]:
test_pool.sort_values(['query_id', 'relevance'], ascending=[True, False])[['query_id', 'org_id']].to_csv('baseline.csv', index=None)